In [8]:
pip install pandas neo4j openpyxl

In [14]:
import pandas as pd
import os
from neo4j import GraphDatabase

# Dictionnaire pour mapper les colonnes attendues aux colonnes réelles
expected_columns = {
    'Entité de formation': 'entite_formation',
    'Etablissements': 'etablissements',
    'Filières de formation': 'filieres_formation',
    'Mode d’entrée': 'mode_entree',
    'Baccalauréat recommandé': 'bac_recommande',
    'Matières': 'matieres',
    'Débouchés /Métiers': 'metiers',
    'Statut': 'statut'
}

# Connexion à Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"  # Remplacez par votre nom d'utilisateur Neo4j
password = "iwadiiwadi"  # Remplacez par votre mot de passe Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))

# Fonction pour créer les nœuds dans Neo4j
def create_graph_from_excel(df):
    with driver.session() as session:
        for index, row in df.iterrows():
            # Vérification que les colonnes existent
            entite_formation = row.get(expected_columns['Entité de formation'], "Non spécifié")
            etablissements = row.get(expected_columns['Etablissements'], "Non spécifié")
            filieres_formation = row.get(expected_columns['Filières de formation'], "Non spécifié")
            mode_entree = row.get(expected_columns['Mode d’entrée'], "Non spécifié")
            bac_recommande = row.get(expected_columns['Baccalauréat recommandé'], "Non spécifié")
            matieres = row.get(expected_columns['Matières'], "Non spécifié")
            metiers = row.get(expected_columns['Débouchés /Métiers'], "Non spécifié")
            statut = row.get(expected_columns['Statut'], "Non spécifié")
            
            # Création des nœuds et relations dans Neo4j
            query = """
            MERGE (e:Entite {nom: $entite_formation})
            MERGE (etab:Etablissement {nom: $etablissements, statut: $statut})
            MERGE (etab)-[:APPARTIENT_A]->(e)
            MERGE (f:Filiere {nom: $filieres_formation, mode_entree: $mode_entree, bac_recommande: $bac_recommande, matieres: $matieres, metiers: $metiers})
            MERGE (etab)-[:PROPOSE]->(f)
            MERGE (f)-[:A_COMME_DEBOUCHE]->(:Metiers {nom: $metiers})
            """
            # Exécution de la requête pour chaque ligne du DataFrame
            session.run(query, entite_formation=entite_formation, etablissements=etablissements, statut=statut, filieres_formation=filieres_formation, 
                        mode_entree=mode_entree, bac_recommande=bac_recommande, matieres=matieres, metiers=metiers)

# Lecture du fichier Excel
file_path = "Extraction.xlsx"
df = pd.read_excel(file_path)
df.columns = df.columns.str.strip()  # Supprimer les espaces au début et à la fin des colonnes

# Appel de la fonction pour créer les noeuds à partir du dataframe
create_graph_from_excel(df)

# Fermer la connexion à Neo4j
driver.close()